In [ ]:
accelerate launch codellama/finetune.py --model_id "codellama/CodeLlama-13b-Instruct-hf" --split "train"  --micro_batch_size 1 --learning_rate 2e-4 --num_proc 2

In [1]:
ls

finetune.py  playground.ipynb


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from transformers import BitsAndBytesConfig
from accelerate import PartialState
checkpoint = "../finetune_codellama2/checkpoint-300"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=getattr(torch, "float16"),
        bnb_4bit_use_double_quant=False,
    )

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

model = AutoModelForCausalLM.from_pretrained("../finetune_codellama2/checkpoint-300", quantization_config=bnb_config,device_map={"": PartialState().process_index})
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, pad_token_id=tokenizer.eos_token_id, max_length=10000, use_cache=True, do_sample=True, temperature=0.9, num_return_sequences=1, clean_up_tokenization_spaces=False)

2024-04-02 05:13:15.143005: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 05:13:15.143034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 05:13:15.144024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 05:13:15.148808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 05:13:15.703085: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("life2scenario-llm24/Life2Scenario-medium")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [4]:
dataset['test'][0]

{'request': 'i would like to ask you to remove pedestrian actor named pedestrian_close_to_standing_16548?\n```\n<?xml version="1.0" ?>\n<OpenSCENARIO>\n  <FileHeader revMajor="1" revMinor="0" date="2020-03-24T12:00:00" description="CARLA:LaneChangeSimple" author=""/>\n  <ParameterDeclarations/>\n  <CatalogLocations/>\n  <RoadNetwork>\n    <LogicFile filepath="Town04"/>\n    <SceneGraphFile filepath=""/>\n  </RoadNetwork>\n  <Entities>\n    <ScenarioObject name="hero">\n      <Vehicle name="vehicle.tesla.model3" vehicleCategory="car">\n        <ParameterDeclarations/>\n        <Performance maxSpeed="69.444" maxAcceleration="200" maxDeceleration="10.0"/>\n        <BoundingBox>\n          <Center x="1.5" y="0.0" z="0.9"/>\n          <Dimensions width="2.1" length="4.5" height="1.8"/>\n        </BoundingBox>\n        <Axles>\n          <FrontAxle maxSteering="0.5" wheelDiameter="0.6" trackWidth="1.8" positionX="3.1" positionZ="0.3"/>\n          <RearAxle maxSteering="0.0" wheelDiameter="0.

In [5]:
import pandas as pd
test = pd.DataFrame(dataset['test'])
test = test[["request", "response"]]

In [6]:
test.head()

,request,response
0,i would like to ask you to remove pedestrian a...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
1,i would like to command you to add pedestrian ...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
2,i would like to lead you to add pedestrian clo...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
3,i would like to order you to remove pedestrian...,"Here is the result:\n```\n<?xml version=""1.0"" ..."
4,i would like to order you to add pedestrian at...,"Here is the result:\n```\n<?xml version=""1.0"" ..."


In [10]:
import sys
sys.path.append('/home/mkbashar/Downloads/nlp/project/life2scenario_core')

from dataset_process import preprocess
from dataset_process import postprocess

def gen(test):
    for i in range(len(test)):
        prompt = test.iloc[3]["request"]
        target = test.iloc[3]["response"]
        
        mod_prompt2 = prompt.split("```")[1].strip()#.split('```')[0]
        prepro = preprocess.DataPreProcess(mod_prompt2)
        mod_prompt = prepro.data_preprocess()
        mod_prompt = prompt.split("```")[0].strip() + "\n```\n" + mod_prompt + "```"

        print ("Starting generation...")
        
        output = pipe(mod_prompt)[0]['generated_text']
        print (output)

        mod_output = output.split("Answer:")[1].split("Answer:")[0]
        postpro = postprocess.DataPostProcess(mod_prompt2, mod_output)
        output = postpro.data_postprocess()

        print ("Starting generation...")

        # output = pipe(prompt)
        with open(f"example/prompt_{i}_check.xml", "w") as f:
            f.write(prompt)
        with open(f"example/target_{i}_check.xml", "w") as f:
            f.write(target)
        with open(f"example/output_{i}_check.xml", "w") as f:
            f.write(output)

        break

gen(test)

Starting generation...
i would like to order you to remove pedestrian close to standing?
```
<?xml version="1.0" ?>
<OpenSCENARIO>
  <FileHeader revMajor="1" revMinor="0" date="2020-03-24T12:00:00" description="CARLA:LaneChangeSimple" author=""/>
  <ParameterDeclarations/>
  <CatalogLocations/>
  <RoadNetwork>
    <LogicFile filepath="Town04"/>
    <SceneGraphFile filepath=""/>
  </RoadNetwork>
  <Entities>
    <ScenarioObject name="hero">
      <Vehicle name="vehicle.tesla.model3" vehicleCategory="car">
        <ParameterDeclarations/>
        <Performance maxSpeed="69.444" maxAcceleration="200" maxDeceleration="10.0"/>
        <BoundingBox>
          <Center x="1.5" y="0.0" z="0.9"/>
          <Dimensions width="2.1" length="4.5" height="1.8"/>
        </BoundingBox>
        <Axles>
          <FrontAxle maxSteering="0.5" wheelDiameter="0.6" trackWidth="1.8" positionX="3.1" positionZ="0.3"/>
          <RearAxle maxSteering="0.0" wheelDiameter="0.6" trackWidth="1.8" positionX="0.0" pos

In [16]:
with open("example/prompt_0_check.xml", "r") as f:
    prompt = f.read()
    with open("example/buffer.xml", "r") as f:
        buffer = f.read()
        mod_prompt2 = prompt.split("```")[1].strip()
        mod_output = buffer.split("Answer:")[1].split("Answer:")[0].split("```")[1].strip().split("```")[0]
        postpro = postprocess.DataPostProcess(mod_prompt2, mod_output)
        output = postpro.data_postprocess()
        with open(f"example/output_0_check2.xml", "w") as f:
            f.write(output)

AttributeError: 'NoneType' object has no attribute 'insert'

In [33]:
s = "Here is the result:\n```\n<?xml version='1.0' ?>\n<OpenSCENARIO>\ndf```\n"

s.split("Here is the result:")[1].split("```")[1].strip()

"<?xml version='1.0' ?>\n<OpenSCENARIO>\ndf"

# Command for Finetuning and Resuming Training of Code-llama

In [ ]:
!accelerate launch codellama/finetune.py --model_id codellama/CodeLlama-13b-Instruct-hf --micro_batch_size 2 --learning_rate 2e-4 --num_proc 2 --streaming 1 --output_dir finetune_codellama23 --max_steps 300

In [ ]:
!accelerate launch codellama/finetune.py --model_id life2scenario_core/finetune_codellama/checkpoint-30 --micro_batch_size 2 --learning_rate 2e-4 --num_proc 2 --streaming 1 --output_dir finetune_codellama --max_steps 400 --resume 1